<a href="https://colab.research.google.com/github/zideric/colab/blob/main/Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dropout

questa tecnica di riduzione dell'overfitting consiste nell'escludere ad ogni esecuzione in modo casuale alcuni neuroni in modo da impedire che il modello memorizzi il set di training (detta in parole poverissime)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense

from keras.callbacks import History

from keras import optimizers

from time import time

## Carichiamo il dataset 

In questo notebook addestreremo una rete neurale in grado di comprendere se una recensione relativa a un film è positiva o negativa. A questo scopo utilizzeremo l'IMDB Moview Review Dataset un dataset di 50.000 recensioni etichettate come positive o negative.
Possiamo caricare il dataset utilizzando direttamente Keras, il parametro num_words ci serve per impostare il numero massimo di parole più frequenti da selezionare e di conseguenza il numero di features del nostro modello.


In [2]:
from keras.datasets import imdb

(X_train, y_train),(X_test, y_test) = imdb.load_data(num_words=5000)

print("Numero di esempi nel train set: %d" % len(X_train))
print("Numero di esempi nel test set: %d" % len(X_test))

17465344/17464789 [==============================] - 0s 0us/step


<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


Numero di esempi nel train set: 25000
Numero di esempi nel test set: 25000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


Le recensioni sono già codificate in delle liste di id, un id identifica la posizione della relativa parola all'interno della lista ordinata delle parole che compaiono più spesso all'interno del nostro corpus di testo, con un offset di 3 posizioni. L'offset è utilizzato perché le prime 3 posizioni sono utilizzate per boh.

In [3]:
print("La prima recensione del train set contiene %d parole" % len(X_train[0]))
print("gli ID delle prime 10 parole della recensione: %s" % X_train[0][:10])

La prima recensione del train set contiene 218 parole
gli ID delle prime 10 parole della recensione: [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]


la seconda parola della prima recensione ha id 14, quindi corrisponde all'11 parola più frequente all'interno del corpus. Possiamo ottenere la posizione di una parola all'interno della lista delle parole più frequenti tramite il dizionario word_index

In [4]:
word_index = imdb.get_word_index()
word_index['love']-3

1646592/1641221 [==============================] - 0s 0us/step


113

La parola 'love' è la 113esima parola più frequente all'interno del nostro corpus (nota l'offset di 3). Per poter ricostruire una frase partendo dagli IDs dobbiamo conoscere la relazione inversa, cioè a quale parola è associato un determinato id.

In [5]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
reverse_word_index.get(113+3) #i primi tre sono riservati

'love'


Adesso possiamo utilizzare il dizionario con la relazione parola->id per ricostruire una frase, facciamolo con la prima frase all'interno del set di addestramento.

In [6]:
decoded_review = [reverse_word_index.get(i -3, '?') for i in X_train[0]]
decoded_review = ' '.join(decoded_review)

print("REVIEW: "+ decoded_review)
print("\n")
print("SENTIMENT [1=Positive/0=Negative]: %d" % y_train[0])

REVIEW: ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly ? was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little ? that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big ? for the whole film but these children are amazing and should be ? for what they have done don't you think the whole s

## Preprocessing 
Le frasi nel corpus di testo sono già codificate in numeri, ma hanno una lunghezza differente, per poterle utilizzare per addestrare la nostra rete neurale abbiamo bisogno di codificarle per ottenere un numero di features consistente. Abbiamo diverse tecniche per farlo, utilizziamo la più semplice: il one hot encoding.

In [7]:
def onehot_encoding(data, size):
  onehot = np.zeros((len(data), size))
  for i, d in enumerate(data):
    onehot[i,d] = 1.
  return onehot

La funzione definita prende in input un corpus di testo e per ogni frase crea un numero di variabili di comodo pari al numero di parole totale del corpus.

In [8]:
X_train = onehot_encoding(X_train, 5000)
X_test = onehot_encoding(X_test, 5000)

X_train.shape

(25000, 5000)

Potremmo ottenere risultati migliori utilizzando tecniche più complesse ottimizzate per il natural language processing (leggasi words embedding) ma per il momento non complichiamoci troppo la vita e passiamo alla creazione del nostro modello.

## Creaiamo la rete neurale

Creiamo un modello di rete neurale profonda con ben 5 strati, 1 di input, 3 nascosti e 1 di output. Trattandosi di un problema di classificazione binaria (positiva/negativa) come funzione di attivazione dell'ultimo strato usiamo la sigmoide e come funzione di costo la binary crossentropy

In [12]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(5000,)))
model.add(Dense(128,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(8,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

il nostro set di addestramento è rappresentato da una matrice spasra, ogni esempio ha 5000 colonne ma soltanto qualche dozzine di parole, proviamo quindi ad utilizzare la funzione di ottimizzazione adamax che in questi casi dovrebbe portare a risultati migliori

In [13]:
model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])

avviamo l'addestarmento per soltanto 10 epoche

In [14]:
model.fit(X_train, y_train, epochs=10, batch_size=512)

Epoch 1/10
49/49 [==============================] - 4s 66ms/step - loss: 0.5356 - accuracy: 0.7106
Epoch 2/10
49/49 [==============================] - 3s 66ms/step - loss: 0.2284 - accuracy: 0.9138
Epoch 3/10
49/49 [==============================] - 3s 67ms/step - loss: 0.1730 - accuracy: 0.9402
Epoch 4/10
49/49 [==============================] - 3s 66ms/step - loss: 0.1268 - accuracy: 0.9624
Epoch 5/10
49/49 [==============================] - 3s 68ms/step - loss: 0.0826 - accuracy: 0.9804
Epoch 6/10
49/49 [==============================] - 3s 67ms/step - loss: 0.0508 - accuracy: 0.9916
Epoch 7/10
49/49 [==============================] - 3s 67ms/step - loss: 0.0283 - accuracy: 0.9967
Epoch 8/10
49/49 [==============================] - 3s 66ms/step - loss: 0.0183 - accuracy: 0.9982
Epoch 9/10
49/49 [==============================] - 3s 66ms/step - loss: 0.0147 - accuracy: 0.9983
Epoch 10/10
49/49 [==============================] - 3s 66ms/step - loss: 0.0119 - accuracy: 0.9986


i risultati ottenuti sono incredibili... si, c'è overfitting

In [15]:
model.evaluate(X_test,y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.5420 - accuracy: 0.8705


[0.5420125722885132, 0.8705199956893921]

## Applicare il dropout

Cerchiamo di contrastare l'overfitting utilizzando una combinazione di regolarizzazione L2 e Dropout. Possiamo utilizzare il Dropout con Keras aggiungendo un'instanza della classe Dropout come fosse un nuovo strato, questa classe necessità di un unico parametro che raprresenta la percentuale di nodi da mantenere attivi.

In [21]:
from keras.regularizers import l2
from keras.layers import Dropout




model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(5000,), kernel_regularizer=l2(0.1)))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(32,activation='relu',kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))
model.add(Dense(8,activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adamax', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=512)

Epoch 1/100
49/49 [==============================] - 4s 77ms/step - loss: 45.7339 - accuracy: 0.5142
Epoch 2/100
49/49 [==============================] - 4s 77ms/step - loss: 1.6237 - accuracy: 0.6224
Epoch 3/100
49/49 [==============================] - 4s 76ms/step - loss: 1.0079 - accuracy: 0.7192
Epoch 4/100
49/49 [==============================] - 4s 76ms/step - loss: 0.8217 - accuracy: 0.7736
Epoch 5/100
49/49 [==============================] - 4s 77ms/step - loss: 0.7517 - accuracy: 0.7938
Epoch 6/100
49/49 [==============================] - 4s 77ms/step - loss: 0.6929 - accuracy: 0.8200
Epoch 7/100
49/49 [==============================] - 4s 76ms/step - loss: 0.6618 - accuracy: 0.8312
Epoch 8/100
49/49 [==============================] - 4s 78ms/step - loss: 0.6439 - accuracy: 0.8368
Epoch 9/100
49/49 [==============================] - 4s 77ms/step - loss: 0.6248 - accuracy: 0.8412
Epoch 10/100
49/49 [==============================] - 4s 76ms/step - loss: 0.6076 - accuracy: 0.849

ora il modello è riuscito a generalizzare meglio

## Mettiamo all'opera la rete neurale

la seguente funzione prende in input una recensione e la fa processare al modello

In [25]:
from re import sub

def preprocess(review):
  #rimuoviamo l'eventuale punteggiatura
  reveiw = sub(r'[^\w\s]','',review)
  #convertiamo tutto in minuscolo
  review = review.lower()
  #creiamo un array di parole
  review = review.split(" ")

  #array vuoto per inserire gli id
  review_array=[]

  #Iteriamo lungo le parole della recensione
  for word in review:
    #proseguiamo se la parola si trova all'interno della lista di parole del corpus di addestramento
    if word in word_index:
      #estraiamo l'indice della parola
      index = word_index[word]
      #proseguiamo se l'indice è minore o uguale a 5000 (altrimenti non lo abbiamo consdierato nel modello)
      if index < 5000:
        review_array.append(word_index[word]+3)

  # eseguiamo one hot encoding sulla lista di indici
  review_array = onehot_encoding([review_array],5000)

  return review_array


Un valore maggiore dell'output della rete corrisponde ad una recensione maggiormente positiva, scriviamo una semplice funzione per interpretare l'output della rete come il sentiment della recensione.

In [28]:
def prob_to_sentiment(prob):
    
    if(prob>0.9): return "fantastica"
    elif(prob>0.75): return "ottima"
    elif(prob>0.55): return "buona" 
    elif(prob>0.45): return "neutrale"
    elif(prob>0.25): return "negativa"
    elif(prob>0.1): return "brutta"
    else: return "pessima"

In [29]:
review = "what a waste of time and cash.. the movie was pointless. with no flow. no questions answered. just a waste. I never review movies but had to share how bad this was..compared to part 1- 2- and 3.... i don't know what else to say other than how misleading the commercial is.. the commercial was cut and spliced with video and audio that didn't even match what happened in the movie... you have been warned. when the movie was over.. people actually Boo'd. hopefully people will spread the word, and save others from throwing their money away. i know die-hard fans will go and give it a shot, but will be disappointed as well. Sinister was better and actually made you jump quite a few times."

x=preprocess(review)
y=model.predict(x)[0]
print("REVIEW: %s" % review)
print("\n")
print("La recensione è %s [%.6f]" % (prob_to_sentiment(y), y))

REVIEW: what a waste of time and cash.. the movie was pointless. with no flow. no questions answered. just a waste. I never review movies but had to share how bad this was..compared to part 1- 2- and 3.... i don't know what else to say other than how misleading the commercial is.. the commercial was cut and spliced with video and audio that didn't even match what happened in the movie... you have been warned. when the movie was over.. people actually Boo'd. hopefully people will spread the word, and save others from throwing their money away. i know die-hard fans will go and give it a shot, but will be disappointed as well. Sinister was better and actually made you jump quite a few times.


La recensione è pessima [0.057806]


proviamo con la recensione di avengers infinity war

In [30]:

review = "This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be."
x = preprocess(review)
y = model.predict(x)

print("REVIEW: %s" % review)
print("\n")
print("La recesione è %s [%.6f]" % (prob_to_sentiment(y), y))

REVIEW: This movie will blow your mind and break your heart - and make you desperate to go back for more. Brave, brilliant and better than it has any right to be.


La recesione è fantastica [0.936425]
